모델을 가장 처음에 학습하기 위한 코드이다. 이후에 파인튜닝을 진행할 시 태그를 더 추가할 수 없기 때문에 필요한 태그들은 아래 추가한 후 진행해야 된다. 이 코드는 gpu 사용량 때문에 colab에서 실행하지 못했고 backend ai를 사용했다.

#**1. 학습을 위해 필요한 라이브러리를 불러온다.**

In [ ]:
!pip install transformers seqeval[gpu]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 11.7 MB/s 
     |████████████████████████████████| 43 kB 1.1 MB/s 
     |████████████████████████████████| 182 kB 56.8 MB/s 
     |████████████████████████████████| 7.6 MB 51.0 MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16179 sha256=2f459d7bd99be20af2234e8e315933c30b32fb076ecf2b2fc71af6040600b248
  Stored in directory: /root/.cache/pip/wheels/ad/5c/ba/05fa33fa5855777b7d686e843ec07452f22a66a138e290e732
Successfully built seqeval


In [ ]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
print(device)

cuda


In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import ElectraTokenizerFast, ElectraConfig, ElectraForTokenClassification

#**2. 학습을 위한 데이터셋을 불러온다.**
> ***데이터의 경로 입력/수정 필수!***

In [ ]:
import pandas as pd
df = pd.read_csv('/home/work/lesik/data/modified_labeled_fixed_dataset.tsv', sep = '\t', keep_default_na=False)
df.head()

,text,label
0,아~ 제목을 그냥 탄소 아~ 그리고 인간 원리 다중 우주 아~ 이런 제목을 잡았는데...,O O O O O MT_ELEMENT O O O O O O O O O O O O O...
1,그니까 현대 우주론이 사실은 어디까지 왔나,O O O TR_SCIENCE TR_SCIENCE O O O O O O O O
2,그리고 현대 우주론의 딜레마가 무엇인가?,O O TR_SCIENCE TR_SCIENCE O O O O O O O
3,아~ 그런 얘기를 이제 간략하게 다중 우주라는 키워드를 통해서 언급하려고 하고요.,O O O O O O O O O TR_SCIENCE TR_SCIENCE O O O ...
4,그 전에 어~ 어제 별을 보셨지만 저게 어~ 우리가 하늘을 바라볼 때 관찰하는 별들...,O O O O O DT_DAY O O O O O O O O O O O O O O O...


*   첫 학습 이후에 파인튜닝을 진행할 시 태그를 더 추가할 수 없기 때문에
fine_tuning과 test를 진행할 때는 무조건 태그가 동일해야 된다.
*   필요한 태그는 전부 아래 추가한 후 진행해야 된다.

In [ ]:
# Split labels based on whitespace and turn them into a list
labels = [i.split() for i in df['label'].values.tolist()]

# Check how many labels are there in the dataset
#말뭉치 데이터에 포함된 총 태그
unique_labels = set()
for lb in labels:
  [unique_labels.add(i) for i in lb if i not in unique_labels]

# Map each label into its id representation and vice versa
labels_to_ids = {k: v for v, k in enumerate(sorted(unique_labels))}
ids_to_labels = {v: k for v, k in enumerate(sorted(unique_labels))}

#말뭉치에 포함되어 있지 않는 태그들 추가
labels_to_ids['CV_ACT'] = 150
ids_to_labels[150] = 'CV_ACT'

labels_to_ids['CV_STATE'] = 151
ids_to_labels[151] = 'CV_STATE'

print(ids_to_labels)
print(len(ids_to_labels))

{'OGG_HOTEL', 'TI_HOUR', 'QT_MAN_COUNT', 'AF_BUILDING', 'CV_POLICY', 'EV_FESTIVAL', 'AFA_ART_CRAFT', 'MT_CHEMICAL', 'QT_SPEED', 'O', 'LCP_CITY', 'OGG_EDUCATION', 'PT_FRUIT', 'QT_OTHERS', 'FD_ART', 'TR_OTHERS', 'AFW_OTHER_PRODUCTS', 'OGG_LAW', 'CV_PRIZE', 'LCG_MOUNTAIN', 'TMI_PROJECT', 'AFA_VIDEO', 'TR_HUMANITIES', 'CV_BUILDING_TYPE', 'MT_METAL', 'CV_SPORTS_INST', 'AFA_DOCUMENT', 'TMI_SW', 'PT_TYPE', 'TR_MEDICINE', 'PT_OTHERS', 'LCG_RIVER', 'LC_SPACE', 'OGG_SCIENCE', 'CV_SPORTS_POSITION', 'QT_SIZE', 'MT_ROCK', 'TMI_EMAIL', 'CV_CULTURE', 'TMI_SITE', 'AF_MUSICAL_INSTRUMENT', 'TI_MINUTE', 'CV_CLOTHING', 'AM_FISH', 'CV_FOOD_STYLE', 'OGG_MILITARY', 'TI_SECOND', 'AM_MAMMALIA', 'AM_INSECT', 'OGG_MEDIA', 'AFA_PERFORMANCE', 'TM_SPORTS', 'CV_DRINK', 'CV_ART', 'TM_CLIMATE', 'CV_LAW', 'DT_MONTH', 'QT_ORDER', 'CV_CURRENCY', 'AF_WEAPON', 'LCG_ISLAND', 'LCP_COUNTY', 'DT_YEAR', 'OGG_FOOD', 'OGG_ECONOMY', 'QT_TEMPERATURE', 'PS_CHARACTER', 'AM_PART', 'PT_TREE', 'CV_OCCUPATION', 'PS_NAME', 'QT_LENGTH', 'O

In [ ]:
# Let's take a look at how can we preprocess the text - Take first example
text = df['text'].values.tolist()
m_len = 0
for t in text:
    if m_len < len(t):
        m_len = len(t)
        
example = text[1]

print(example)
print(m_len)

#**3. koELECTRA 모델과 토크나이저를 불러온다.**

In [ ]:
model = ElectraForTokenClassification.from_pretrained("monologg/koelectra-small-v2-discriminator", num_labels=len(labels_to_ids))
model.to(device)
tokenizer = ElectraTokenizerFast.from_pretrained('monologg/koelectra-small-v2-discriminator')

Downloading:   0%|          | 0.00/55.1M [00:00<?, ?B/s]

Some weights of the model checkpoint at monologg/koelectra-small-v2-discriminator were not used when initializing ElectraForTokenClassification: ['discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense.weight']
- This IS expected if you are initializing ElectraForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForTokenClassification were not initialized from the model checkpoint at monologg/koelectra-small-v2-discriminator and are newly initialized: ['classifi

#**4. 토큰화를 하기 위해 필요한 코드이다.**
> ***원하는 epoch로 수정 가능!***

epoch 개수는 고정이 아니므로 각 모델에 적절 또는 최적화 되어있는 개수로 변경하면 된다. \
학습 중에 중단 되었을 경우, 저장된 epoch부터 이어서 학습 시킬 수 있다. 단, epochs를 저장된 epoch만큼 빼서 변경해줘야한다.
> ex.) epoch 72를 목표하였고, epoch 48까지 저장된 후 중단 되었다면 \
72-48 = 24; epochs를 24로 변경해주면 된다. 

(7.학습 실행 코드 참고)

In [ ]:
from transformers import ElectraTokenizerFast

MAX_LEN = 256
TRAIN_BATCH_SIZE = 64
VALID_BATCH_SIZE = 64
EPOCHS = 72
LEARNING_RATE = 1e-05
MAX_GRAD_NORM = 10

In [ ]:
class ElectraDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.len = len(dataframe)
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __getitem__(self, index):
        # step 1: get the sentence and word labels 
        sentence = self.data.text[index].strip()
        word_labels = self.data.label[index].split()

        # step 2: use tokenizer to encode sentence (includes padding/truncation up to max length)
        # BertTokenizerFast provides a handy "return_offsets_mapping" functionality for individual tokens
        encoding = self.tokenizer(sentence,
                             return_offsets_mapping=True, 
                             padding='max_length', 
                             truncation=True, 
                             max_length=self.max_len)
        
        valid_token_list = []
        for idx, mapping in enumerate(encoding["offset_mapping"]):
            if mapping[0] == 0 and mapping[1] == 0:
                continue
            valid_token_list.append(mapping)
        if len(valid_token_list) != len(word_labels):
            print(index, len(word_labels), len(valid_token_list), sentence)
        
        # step 3: create token labels only for first word pieces of each tokenized word
        labels = [labels_to_ids[label] for label in word_labels] 
        # code based on https://huggingface.co/transformers/custom_datasets.html#tok-ner
        # create an empty array of -100 of length max_length
        encoded_labels = np.ones(len(encoding["offset_mapping"]), dtype=int) * -100
        
        # set only labels whose first offset position is 0 and the second is not 0
        i = 0
        if len(labels) != 0:
            for idx, mapping in enumerate(encoding["offset_mapping"]):
                if mapping[0] == 0 and mapping[1] == 0:
                    continue
                tok = tokenizer.convert_ids_to_tokens(encoding['input_ids'][idx])
            
                # overwrite label
                if i == len(labels):
                    break
                encoded_labels[idx] = labels[i]
                i += 1
                
        # step 4: turn everything into PyTorch tensors
        item = {key: torch.as_tensor(val) for key, val in encoding.items()}
        item['label'] = torch.as_tensor(encoded_labels)
        
        return item
    def __len__(self):
        return self.len

In [ ]:
train_size = 0.8
train_dataset = df.sample(frac=train_size,random_state=200)
test_dataset = df.drop(train_dataset.index).reset_index(drop=True)
train_dataset = train_dataset.reset_index(drop=True)

print("FULL Dataset: {}".format(df.shape))
print("TRAIN Dataset: {}".format(train_dataset.shape))
print("TEST Dataset: {}".format(test_dataset.shape))

training_set = ElectraDataset(train_dataset, tokenizer, MAX_LEN)
testing_set = ElectraDataset(test_dataset, tokenizer, MAX_LEN)

FULL Dataset: (356504, 2)
TRAIN Dataset: (285203, 2)
TEST Dataset: (71301, 2)


In [ ]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 4
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 4
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [ ]:
inputs = training_set[2]
input_ids = inputs["input_ids"].unsqueeze(0)
attention_mask = inputs["attention_mask"].unsqueeze(0)
labels = inputs["label"].unsqueeze(0)

input_ids = input_ids.to(device)
attention_mask = attention_mask.to(device)
labels = labels.to(device)

outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
initial_loss = outputs[0]
initial_loss

tensor(4.9279, device='cuda:0', grad_fn=<NllLossBackward0>)

In [ ]:
tr_logits = outputs[1]
tr_logits.shape

torch.Size([1, 256, 152])

In [ ]:
!pip install tensorboardX

from tensorboardX import SummaryWriter
writer = SummaryWriter()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 125 kB 13.0 MB/s 


In [ ]:
optimizer = torch.optim.Adam(params=model.parameters(), lr=LEARNING_RATE)

#**5. train, validation 함수를 불러오는 섹션이다.**

train을 위한 함수이다.

In [ ]:
# Defining the training function on the 80% of the dataset for tuning the bert model
def train(epoch):
    tr_loss, tr_accuracy = 0, 0
    nb_tr_examples, nb_tr_steps = 0, 0
    tr_preds, tr_labels = [], []
    # put model in training mode
    model.train()
    
    for idx, batch in enumerate(training_loader):
        ids = batch['input_ids'].to(device, dtype = torch.long)
        mask = batch['attention_mask'].to(device, dtype = torch.long)
        labels = batch['label'].to(device, dtype = torch.long)

        output = model(ids, attention_mask=mask, labels=labels)
        loss = output[0]
        tr_logits = output[1]
        tr_loss += loss.item()

        nb_tr_steps += 1
        nb_tr_examples += labels.size(0)
        
        if idx % 100==0:
            loss_step = tr_loss/nb_tr_steps
            print(f"Training loss per 100 training steps: {loss_step}")
           
        # compute training accuracy
        flattened_targets = labels.view(-1) # shape (batch_size * seq_len,)
        active_logits = tr_logits.view(-1, model.config.num_labels) # shape (batch_size * seq_len, num_labels)
        flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
        
        # only compute accuracy at active labels
        active_accuracy = labels.view(-1) != -100 # shape (batch_size, seq_len)
        #active_labels = torch.where(active_accuracy, labels.view(-1), torch.tensor(-100).type_as(labels))
        
        labels = torch.masked_select(flattened_targets, active_accuracy)
        predictions = torch.masked_select(flattened_predictions, active_accuracy)
        
        tr_labels.extend(labels)
        tr_preds.extend(predictions)

        tmp_tr_accuracy = accuracy_score(labels.cpu().numpy(), predictions.cpu().numpy())
        tr_accuracy += tmp_tr_accuracy
    
        # gradient clipping
        torch.nn.utils.clip_grad_norm_(
            parameters=model.parameters(), max_norm=MAX_GRAD_NORM
        )
        
        # backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    epoch_loss = tr_loss / nb_tr_steps
    tr_accuracy = tr_accuracy / nb_tr_steps
    print(f"Training loss epoch: {epoch_loss}")
    print(f"Training accuracy epoch: {tr_accuracy}")
    writer.add_scalar('Train/Loss', epoch_loss, epoch)
    writer.add_scalar('Train/Accuracy', tr_accuracy, epoch)
    

validation을 위한 함수이다.

In [ ]:
def valid(epoch):
    # put model in evaluation mode
    model.eval()
    
    eval_loss, eval_accuracy = 0, 0
    nb_eval_examples, nb_eval_steps = 0, 0
    eval_preds, eval_labels = [], []
    
    with torch.no_grad():
        for idx, batch in enumerate(testing_loader):
            
            ids = batch['input_ids'].to(device, dtype = torch.long)
            mask = batch['attention_mask'].to(device, dtype = torch.long)
            labels = batch['label'].to(device, dtype = torch.long)
            
            output = model(input_ids=ids, attention_mask=mask, labels=labels)
            loss = output[0]
            eval_logits = output[1]
            
            eval_loss += loss.item()

            nb_eval_steps += 1
            nb_eval_examples += labels.size(0)
        
            if idx % 100==0:
                loss_step = eval_loss/nb_eval_steps
                print(f"Validation loss per 100 evaluation steps: {loss_step}")
              
            # compute evaluation accuracy
            flattened_targets = labels.view(-1) # shape (batch_size * seq_len,)
            active_logits = eval_logits.view(-1, model.config.num_labels) # shape (batch_size * seq_len, num_labels)
            flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
            
            # only compute accuracy at active labels
            active_accuracy = labels.view(-1) != -100 # shape (batch_size, seq_len)
        
            labels = torch.masked_select(flattened_targets, active_accuracy)
            predictions = torch.masked_select(flattened_predictions, active_accuracy)
            
            eval_labels.extend(labels)
            eval_preds.extend(predictions)
            
            tmp_eval_accuracy = accuracy_score(labels.cpu().numpy(), predictions.cpu().numpy())
            eval_accuracy += tmp_eval_accuracy

    labels = [ids_to_labels[id.item()] for id in eval_labels]
    predictions = [ids_to_labels[id.item()] for id in eval_preds]
    
    eval_loss = eval_loss / nb_eval_steps
    eval_accuracy = eval_accuracy / nb_eval_steps
    print(f"Validation Loss: {eval_loss}")
    print(f"Validation Accuracy: {eval_accuracy}")
    writer.add_scalar('Validation/Loss', eval_loss, epoch)
    writer.add_scalar('Validation/Accuracy', eval_accuracy, epoch)

    return labels, predictions

#**6. save 함수를 불러오는 섹션이다.**
> ***directory/model/tokenizer 이름 변경은 필수!***

In [ ]:
import os
def save(epoch):
    directory = "/home/work/lesik/model/FIXED_EPOCH_"+str(epoch)

    if not os.path.exists(directory):
        os.makedirs(directory)
    model.save_pretrained(directory)

    torch.save(model.state_dict(), directory+"/model.pt")
    directory = "/home/work/lesik/tokenizer/FIXED_EPOCH_" + str(epoch)
    if not os.path.exists(directory):
        os.makedirs(directory)
    # save vocabulary of the tokenizer
    tokenizer.save_vocabulary(directory)
    tokenizer.save_pretrained(directory)
    # save the model weights and its configuration file
    print('All files saved')
    print('This tutorial is completed')

#**7. 학습을 실행하는 코드이다.**
> ***학습 과정에서 끊겼을 경우, prev_epoch 변경 필수! 그 외는 0으로 실행!***


*   prev_epoch는 학습을 시작하는 지점을 뜻하는 epoch이다.
*   학습하다 끊겼을 경우, 저장 단위의 배수를 계산하여 마지막으로 저장된 epoch로 변경 해주면 된다. \
또한, 토큰화에서도 목표하고자 하는epoch를 저장된 epoch만큼 빼서 변경해줘야 한다.
> ex.)epoch가 50에서 중단 되었을 경우, \
epoch 48까지 저장되었기 때문에 prev_epoch는 48로 시작. 토큰화의 epoch는 24로 변경. \
단, 51에서 중단되었을 경우, \
epoch 51이 저장되었다면, prev_epoch는 51부터 시작. 토큰화의 epoch는 21로 변경. \
epoch 51이 저장되지 않았다면,prev_epoch는 48부터 시작. 토큰화의 epoch는 24로 변경.
*   epoch는 현재 3의 배수로 저장되고 있으며, 변경이 가능하다.

In [ ]:
prev_epoch = 0                   #학습을 시작하는 epoch
for epoch in range(prev_epoch + 1, prev_epoch + 1 + EPOCHS):
    print(f"epoch: {epoch}")
    train(epoch)
    valid(epoch)
    if epoch != 0 and epoch % 3 == 0:     #현재 3의 배수로 저장되고 있으며 변경 가능 (3을 변경해주면 됩니다.)
        save(epoch)

epoch: 1


/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Training loss per 100 training steps: 4.74533224105835
Training loss per 100 training steps: 3.736000901401633
Training loss per 100 training steps: 3.310012743840763
Training loss per 100 training steps: 3.085151396716552
Training loss per 100 training steps: 2.9270929267578887
Training loss per 100 training steps: 2.798927214807141


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py", line 3326, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-23-c21357dbc8f3>", line 4, in <module>
    train(epoch)
  File "<ipython-input-19-c25f3af8c943>", line 51, in train
    loss.backward()
  File "/usr/local/lib/python3.8/dist-packages/torch/_tensor.py", line 487, in backward
    torch.autograd.backward(
  File "/usr/local/lib/python3.8/dist-packages/torch/autograd/__init__.py", line 197, in backward
    Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py", line 2040, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'KeyboardInterrupt' object has no attribute '_r

KeyboardInterrupt: ignored

In [ ]:
writer.close()